In [1]:
import torch
from torch.utils.data import Dataset
from PIL import Image
import pandas as pd
import os

class RetinaDataset(Dataset):
    def __init__(self, csv_path, img_dir, transform=None):
        self.data = pd.read_csv(csv_path)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        
        img_id = row["ID"]
        label = row["Disease_Risk"]   # 0 = healthy, 1 = unhealthy

        img_path = os.path.join(self.img_dir, f"{img_id}.png")
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label, dtype=torch.long)

In [2]:
from torchvision import transforms

train_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor()
])

train_csv = "data/Training_Set/Training_Set/RFMID_Training_Labels.csv"
train_img_dir = "data/Training_Set/Training_Set/Training/"

train_dataset = RetinaDataset(train_csv, train_img_dir, transform=train_transform)


In [3]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)


In [4]:
import torch.nn as nn
import torch.nn.functional as F

class BasicCNN(nn.Module):
    def __init__(self):
        super(BasicCNN, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)

        self.pool = nn.MaxPool2d(2, 2)
        
        self.fc1 = nn.Linear(64 * 28 * 28, 128)
        self.fc2 = nn.Linear(128, 2)   # 2 sınıf: Healthy, Unhealthy

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        
        x = x.view(-1, 64 * 28 * 28)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BasicCNN().to(device)
model

BasicCNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=50176, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=2, bias=True)
)

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [7]:
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    running_loss = 0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")


Epoch [1/5], Loss: 0.4925
Epoch [2/5], Loss: 0.4519
Epoch [3/5], Loss: 0.4350
Epoch [4/5], Loss: 0.4216
Epoch [5/5], Loss: 0.4127


In [8]:
# Test CSV ve resim klasörü yolları
test_csv = "data/Test_Set/Test_Set/RFMID_Testing_Labels.csv"
test_img_dir = "data/Test_Set/Test_Set/Test/"

test_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor()
])

test_dataset = RetinaDataset(test_csv, test_img_dir, transform=test_transform)

test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
len(test_dataset)


640

In [9]:
correct = 0
total = 0
model.eval()

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print("Test Accuracy: {:.2f}%".format(accuracy))


Test Accuracy: 86.88%


In [12]:
from sklearn.metrics import confusion_matrix, f1_score, roc_auc_score
import torch.nn.functional as F

model.eval()

all_labels = []
all_preds = []
all_probs = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)

        # Tahmin edilen sınıf (0/1)
        preds = torch.argmax(outputs, dim=1)

        # Unhealthy (1) için olasılık
        probs = F.softmax(outputs, dim=1)[:, 1]

        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())
        all_probs.extend(probs.cpu().numpy())

#  METRİKLER
cm = confusion_matrix(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds)
roc_auc = roc_auc_score(all_labels, all_probs)

#  YAZDIR 
print("Sonuçlar:")
print("F1 Score     :", round(f1, 4))
print("ROC-AUC      :", round(roc_auc, 4))
print("Confusion Matrix:\n", cm)


Sonuçlar:
F1 Score     : 0.9195
ROC-AUC      : 0.8726
Confusion Matrix:
 [[ 76  58]
 [ 26 480]]


In [13]:
# Eğitilmiş modeli kaydet
torch.save(model.state_dict(), "basiccnn_model.pth")

print("Model başarıyla kaydedildi.")


Model başarıyla kaydedildi.
